In [10]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split

In [26]:
df = pd.read_csv("data_edited.csv")

In [27]:
x_data = df[["Close-Direction-Delay0","Close-Direction-Delay1","Close-Direction-Delay2","Close-Direction-Delay3",
            "Close-Direction-Delay4","Close-Direction-Delay5","Close-Direction-Delay6","Close-Direction-Delay7",
            "Close-Direction-Delay8","Close-Direction-Delay9"]].values

y_data = df["Close-direction"].values

x_data = torch.Tensor(x_data)
x_data = torch.unsqueeze(x_data, dim=1)

y_data = torch.Tensor(y_data)
y_data = torch.unsqueeze(y_data, dim=1)

In [28]:
mps_device = torch.device("mps")
# Model tanimla
model = torch.nn.Sequential(
    torch.nn.Linear(10, 250),
    torch.nn.ReLU(),
    torch.nn.Linear(250, 250),
    torch.nn.Sigmoid(),
    torch.nn.Linear(250, 1),
)

# Kayip fonksiyonu ve optimizer tanimla
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
losses = []
epochs = 1000
# Egitim dongusu
for epoch in range(epochs):
    # Tahmini hesapla
    y_pred = model(x_data)

    # Kaybi hesapla
    loss = loss_fn(y_pred, y_data)
    
    losses.append(loss.detach().numpy())

    # Gradyanlari geriye yay
    optimizer.zero_grad()
    loss.backward()

    # Ağırlıkları güncelle
    optimizer.step()
    if(loss < 0.001):
        break
    if(epoch%10==0):
        print(f"Epoch: {epoch} and loss: {loss}")

/Users/haluk/opt/miniconda3/envs/TraderAI/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4203, 1])) that is different to the input size (torch.Size([4203, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0 and loss: 1.5095199346542358
Epoch: 10 and loss: 1.0787760019302368
Epoch: 20 and loss: 0.7507278919219971
Epoch: 30 and loss: 0.522734522819519
Epoch: 40 and loss: 0.38099586963653564
Epoch: 50 and loss: 0.30435436964035034
Epoch: 60 and loss: 0.2697678804397583
Epoch: 70 and loss: 0.2575775384902954
Epoch: 80 and loss: 0.254630982875824
Epoch: 90 and loss: 0.2542895972728729
Epoch: 100 and loss: 0.25424569845199585
Epoch: 110 and loss: 0.2541021406650543
Epoch: 120 and loss: 0.2539280951023102
Epoch: 130 and loss: 0.2537755072116852
Epoch: 140 and loss: 0.25364160537719727
Epoch: 150 and loss: 0.2535136044025421
Epoch: 160 and loss: 0.25338640809059143
Epoch: 170 and loss: 0.25325995683670044
Epoch: 180 and loss: 0.25313499569892883
Epoch: 190 and loss: 0.25301191210746765
Epoch: 200 and loss: 0.2528908848762512
Epoch: 210 and loss: 0.2527720630168915
Epoch: 220 and loss: 0.2526557147502899
Epoch: 230 and loss: 0.25254204869270325
Epoch: 240 and loss: 0.25243112444877625
Epo

In [29]:
from torcheval.metrics import R2Score
metric = R2Score()
metric.update(y_pred, y_data)
metric.compute()

ValueError: The dimension `input` and `target` should be 1D or 2D, got shapes torch.Size([4203, 1, 1]) and torch.Size([4203, 1]).